# Interactive Mapping 

In [1]:
# Load packages 
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns         
import numpy as np
import os
import folium
import plotly.express as px
import rasterio 
import rioxarray as rioxr
import json
# from bokeh.plotting import figure, show

# import libpysal as lps 
# from libpysal.weights import W 
# from esda.getisord import G_Local
# from functions import state_gstar

# Remove max columns
pd.set_option('display.max_columns', None)

In [2]:
# Read in GeoJSON of all 
national_results = gpd.read_file('/capstone/justice40/data/national_results.geojson')
national_results.head()

,tract_id,total_criteria,total_burdens,dac_wo_neighbors,dac_relaxed_low_income,dac,percent_area_dac,share_of_dac_neighbors,total_population,original_g_star,standardized_g_star,p_norm,standardized_g_star_crit,p_norm_crit,geometry
0,1001020100,0,0.0,False,False,False,0,0.0,1993.0,0.0,-0.620617,0.267426,-0.560448,0.287587,"POLYGON ((-86.48196 32.49876, -86.48189 32.498..."
1,1001020200,0,0.0,False,False,False,0,0.0,1959.0,0.0,-0.620617,0.267426,-0.560448,0.287587,"POLYGON ((-86.46828 32.4737, -86.468 32.47371,..."
2,1001020300,0,0.0,False,False,False,0,0.0,3507.0,0.0,-0.620617,0.267426,-0.560448,0.287587,"POLYGON ((-86.46538 32.47372, -86.46597 32.473..."
3,1001020400,0,0.0,False,False,False,0,0.0,3878.0,0.0,-0.620617,0.267426,-0.560448,0.287587,"POLYGON ((-86.43807 32.45349, -86.43812 32.453..."
4,1001020500,0,0.0,False,False,False,0,0.0,10596.0,0.0,-0.620617,0.267426,-0.560448,0.287587,"POLYGON ((-86.4118 32.47416, -86.4118 32.47369..."


In [3]:
#national_results['total_burdens'].plot(kind='hist', bins=8)
#plt.title('Histogram of Total Burdens')

In [4]:
#national_results['total_criteria'].plot(kind='hist')
#plt.title('Histogram of Total Criteria')

In [5]:
# national_results.crs

In [6]:
# west = -124.848974  # Leftmost longitude (Cape Alava, Washington)
# east = -66.947028   # Rightmost longitude (West Quoddy Head, Maine)
# south = 24.396308   # Bottommost latitude (Key West, Florida)
# north = 49.384358   # Topmost latitude (Northwest Angle, Minnesota)

# bbox = [west, south, east, north]  # [min_lon, min_lat, max_lon, max_lat]
# # or as
# bbox = [[south, west], [north, east]]  # [[min_lat, min_lon], [max_lat, max_lon]]

# map = folium.Map()
# map.fit_bounds(bbox)
# map

In [7]:
# folium.Choropleth(
#     geo_data=national_results,
#     name="choropleth",
#     data=national_results,
#     columns=["tract_id", "standardized_g_star"],
#     key_on="feature.properties.tract_id",  # Changed this line
#     fill_color="RdBu",
#     fill_opacity=0.7,
#     line_opacity=0.2,
#     legend_name="Standardized G Star",
# ).add_to(map)


In [8]:
# folium.LayerControl().add_to(map)
# map

In [9]:
# national_small = national_results[['geometry', 'standardized_g_star']]
# national_small = national_small.rename(columns={'standardized_g_star':'stand_gstar'})
# national_small.head()

In [10]:
# len(national_small)

In [11]:
# national_small.to_file('/capstone/justice40/data/national_small.geojson', driver="GeoJSON")  

In [12]:
# gstar_test = national_results[['tract_id', 'standardized_g_star']]
# gstar_test = gstar_test.rename(columns={'standardized_g_star':'stand_gstar'})
# gstar_test.head()

In [13]:
# gstar_test.to_csv('/capstone/justice40/data/gstar_test.csv', index=False)


In [14]:
national_results = national_results[national_results.geometry.notnull()]


In [15]:
national_results["geometry"] = national_results.simplify(0.001)


In [ ]:
# Center the map around the geometries in your GeoDataFrame
center = national_results.geometry.centroid.unary_union.centroid
latitude, longitude = center.y, center.x

# Initialize the map
map = folium.Map(location=[latitude, longitude], zoom_start=6)

# Add the choropleth layer
folium.Choropleth(
    geo_data=national_results,                          # GeoDataFrame automatically converted to GeoJSON
    name="choropleth",
    data=national_results,                              # Use the same GeoDataFrame
    columns=["tract_id", "standardized_g_star"],        # Match your DataFrame columns
    key_on="feature.properties.tract_id",               # Match the GeoJSON property
    fill_color="RdBu_r",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Standardized G Star",
).add_to(map)

# Add layer control
folium.LayerControl().add_to(map)

# Save or display the map
map.save("index.html")
map



/tmp/ipykernel_1038415/882456021.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center = national_results.geometry.centroid.unary_union.centroid
/tmp/ipykernel_1038415/882456021.py:2: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  center = national_results.geometry.centroid.unary_union.centroid


In [ ]:
# print(national_results.iloc[0])
